## Setup: Finnhub API Key

This notebook uses real market data from [Finnhub](https://finnhub.io/) (free tier: 60 API calls/min).

**To use real data:**
1. Get a free API key at https://finnhub.io/register
2. Set environment variable: `export FINNHUB_API_KEY=your_key_here`
3. Or enter the key when prompted

**Without a key:** The notebook falls back to synthetic data generation.


# Market Making Strategy

We simulate a market maker quoting around a midprice with constant spread.

Inventory control is used to manage risk exposure.

Profit & loss is computed with simulated fills.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Import Finnhub helper
try:
    from python.finnhub_helper import fetch_historical_simulation, get_finnhub_api_key
    FINNHUB_AVAILABLE = True
except Exception as e:
    print(f"Finnhub helper not available: {e}")
    FINNHUB_AVAILABLE = False

# Get API key
api_key = get_finnhub_api_key() if FINNHUB_AVAILABLE else None

if FINNHUB_AVAILABLE and api_key:
    print("Fetching real market data from Finnhub...")
    symbols = ['BINANCE:BTCUSDT']
    df = fetch_historical_simulation(symbols, periods=300, api_key=api_key)
    mid_price = df['BINANCE:BTCUSDT_mid'].values
    print(f"Fetched {len(mid_price)} price points")
    print(f"BTC price range: ${mid_price.min():.2f} - ${mid_price.max():.2f}")
else:
    print("Using synthetic data")
    T = 300
    mid_price = np.cumsum(np.random.randn(T)) + 100

T = len(mid_price)
spread = 0.5
bid = mid_price - spread / 2
ask = mid_price + spread / 2

inventory = np.zeros(T)
pnl = np.zeros(T)
cash = 0.0

for t in range(1, T):
    # Simulate random fills: buy on bid, sell on ask
    fill = np.random.choice([-1, 1])  # -1 = buy (inventory up), 1 = sell (inventory down)
    
    inventory[t] = inventory[t-1] + fill
    
    # When buying (-1): pay ask price (cash decreases)
    # When selling (1): receive bid price (cash increases)
    if fill == -1:
        cash -= ask[t]
    else:
        cash += bid[t]
    
    # Mark-to-market PnL: cash + inventory * mid_price
    pnl[t] = cash + inventory[t] * mid_price[t]

print(f"Final inventory: {inventory[-1]:.2f}")
print(f"Final PnL: ${pnl[-1]:.2f}")
print(f"Number of trades: {T-1}")

plt.figure(figsize=(12, 8))
plt.subplot(3,1,1)
plt.plot(mid_price, label='Mid Price')
plt.plot(bid, '--', alpha=0.5, label='Bid')
plt.plot(ask, '--', alpha=0.5, label='Ask')
plt.legend()
plt.title('Market Making Prices')

plt.subplot(3,1,2)
plt.plot(inventory)
plt.axhline(0, color='r', linestyle='--', alpha=0.3)
plt.title('Inventory Over Time')

plt.subplot(3,1,3)
plt.plot(pnl)
plt.title('PnL Over Time')

plt.tight_layout()
plt.show()
